In [1]:
#os
import os

#ml
import numpy as np
import torch
import mmcv
import cv2
from mmdet.apis    import init_detector
from mmrotate.apis import inference_detector_by_patches
from dataclasses import asdict
import json

from geo_util import Pnt, Vehicle, VehicleExport, rbbox_to_poly

In [2]:

def process_results( result, conf_thr: float ):
    if isinstance(result, tuple):
        bbox_result, segm_result = result
        if isinstance( segm_result, tuple ):
            segm_result = segm_result[0]
    else:
        bbox_result, segm_result = result, None
    bboxes = np.vstack(bbox_result)
    labels = [
        np.full( bbox.shape[0], i, dtype=np.int32 )
        for i, bbox in enumerate(bbox_result)
    ]
    labels = np.concatenate( labels )
    #remove everything that isn't a small vehicle    
    # Apply both filters together
    conf_mask = bboxes[:, 5] >= conf_thr
    class_mask = (labels == 4) | (labels == 5)
    combined_mask = conf_mask & class_mask
    bboxes = bboxes[combined_mask]
    labels = labels[combined_mask]
    # print( f"Detected {len( bboxes )} vehicles." )
    reshaped = labels.reshape( -1, 1 )
    concat = np.concatenate( (bboxes, reshaped), axis=1 )
    a_entries = [Vehicle( r[0], r[1], r[2], r[3], r[4], r[5], r ) for r in concat]
    vas = [VehicleExport( Pnt( a.x, a.y ), a.width, a.height, a.theta, a.confidence, rbbox_to_poly( a.arr ).tolist(), str(i) ) for i, a in enumerate(a_entries)]
    return vas

In [3]:
class InitArgs:
    def __init__(self):
        self.device     = 'cuda:0'
        self.config     = "mmrotate/configs/redet/redet_re50_refpn_1x_dota_ms_rr_le90.py"
        self.checkpoint = "model/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth"

class Args:
    def __init__( self, batch_size: int, patch_size: int, patch_step: int, img: str, conf: float ):
        self.img = img
        self.score_thr = conf
        self.merge_iou_thr = 0.85
        self.img_ratios = [1.0]
        self.batch_size = batch_size
        self.patch_sizes = [patch_size]
        self.patch_steps = [patch_step]
        self.palette = 'dota'

torch.cuda.set_per_process_memory_fraction( 0.75 )

In [4]:
init_args = InitArgs()
try:
    model
except NameError:
    model = init_detector( init_args.config, init_args.checkpoint, device=init_args.device )

/home/cory/proj/mmrotate-clean/mmrotate/mmrotate/models/backbones/re_resnet.py:481: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/home/cory/miniconda3/envs/openmmlab-clean/lib/python3.11/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)
/home/cory/miniconda3/envs/openmmlab-clean/lib/python3.11/site-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: model/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: backbone.conv1.filter, backbone.layer2.0.conv1.filter, backbone.layer2.0.conv2.filter, backbone.layer2.0.conv3.filter, backbone.layer2.0.downsample.0.filter, backbone.layer2.1.conv1.filter, backbone.layer2.1.conv2.filter, backbone.layer2.1.conv3.filter, backbone.layer2.2.conv1.filter, backbone.layer2.2.conv2.filter, backbone.layer2.2.conv3.filter, backbone.layer2.3.conv1.filter, backbone.layer2.3.conv2.filter, backbone.layer2.3.conv3.filter, backbone.layer3.0.conv1.filter, backbone.layer3.0.conv2.filter, backbone.layer3.0.conv3.filter, backbone.layer3.0.downsample.0.filter, backbone.layer3.1.conv1.filter, backbone.layer3.1.conv2.filter, backbone.layer3.1.conv3.filter, backbone.layer3.2.conv1.filter, backbone.layer3.2.conv2.filter, backbone.layer3.2.conv3.filter, backbone.layer3.3.conv1.filter, backbone.layer3.3.co

In [5]:
from pathlib import Path
import numpy as np
from typing import List
from PIL import Image

def load_gt(img_path: str) -> List[VehicleExport]:
    """Load ground truth from corresponding .txt file"""
    gt_path = Path(img_path).with_suffix('.txt')
    if not gt_path.exists():
        print(f"Warning: GT file {gt_path} not found")
        return []
    
    # Get image dims with fallback
    try:
        w, h = Image.open(img_path).size
    except:
        w, h = 1024, 1024
        print(f"Warning: Using default dims for {img_path}")
    
    vehicles = []
    with open(gt_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 6:
                continue
                
            cls, cx_n, cy_n, w_n, h_n, angle = map(float, parts[:6])
            cx, cy, w_abs, h_abs = cx_n * w, cy_n * h, w_n * w, h_n * h
            
            arr = np.array([cx, cy, w_abs, h_abs, angle, 1.0, cls])
            poly = rbbox_to_poly(arr).tolist()
            
            vehicles.append(VehicleExport(
                label=int(cls), center=Pnt(cx, cy), width=w_abs, 
                height=h_abs, theta=angle, polygon=poly, confidence=1.0
            ))
    
    return vehicles

def get_dist(p1, p2):
    """Euclidean distance between points"""
    return np.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

def find_matches(dets: List[VehicleExport], gt: List[VehicleExport], max_dist: float = 10.0):
    """Find nearest GT matches using greedy assignment"""
    matches = []
    used = set()
    
    for i, det in enumerate(dets):
        best_dist, best_j = float('inf'), -1
        
        for j, g in enumerate(gt):
            if j in used:
                continue
            dist = get_dist(det.center, g.center)
            if dist < best_dist and dist <= max_dist:
                best_dist, best_j = dist, j
        
        if best_j != -1:
            matches.append((i, best_j, best_dist))
            used.add(best_j)
    
    return matches

def benchmark(img_path: str, dets: List[VehicleExport]):
    """Simple benchmark using nearest center matching"""
    gt = load_gt(img_path)
    if not gt:
        print(f"No GT for {Path(img_path).name}")
        return
    
    matches = find_matches(dets, gt, 10)
    
    tp = len(matches)
    fp = len(dets) - tp
    fn = len(gt) - tp
    
    prec = tp / len(dets) if dets else 0.0
    rec = tp / len(gt) if gt else 0.0
    f1 = 2 * prec * rec / (prec + rec) if prec + rec else 0.0
    
    name = Path(img_path).name
    print(f"\n{name}:")
    print(f"Dets: {len(dets)}, GT: {len(gt)}")
    print(f"Prec: {prec:.3f}, Rec: {rec:.3f}, F1: {f1:.3f}")
    print(f"TP: {tp}, FP: {fp}, FN: {fn}")
    
    return {
        'precision': prec, 'recall': rec, 'f1': f1,
        'tp': tp, 'fp': fp, 'fn': fn, 'matches': matches
    }

In [ ]:
import pathlib as path 
import glob 
dir  = "/mnt/d/Dropbox/ZoneSentinel/Challenge/Input"
imgs = txt_files = glob.glob(  f"{dir}/*.png" )
for img_path in imgs:
    args = Args( batch_size=32, patch_size=384, patch_step=64, img=img_path, conf=0.00 )
    img = mmcv.imread( args.img )
    result = inference_detector_by_patches( model, img,
                                            args.patch_sizes,
                                            args.patch_steps,
                                            args.img_ratios,
                                            args.merge_iou_thr,
                                            args.batch_size )
    
    dets = process_results( result, args.score_thr )
    benchmark( img_path, dets ) 
    # model.show_result( img, result, show=True )
    #print( dets )

/home/cory/miniconda3/envs/openmmlab-clean/lib/python3.11/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '



1.png:
Dets: 27, GT: 18
Prec: 0.667, Rec: 1.000, F1: 0.800
TP: 18, FP: 9, FN: 0

10.png:
Dets: 16, GT: 18
Prec: 0.875, Rec: 0.778, F1: 0.824
TP: 14, FP: 2, FN: 4

11.png:
Dets: 175, GT: 167
Prec: 0.794, Rec: 0.832, F1: 0.813
TP: 139, FP: 36, FN: 28

12.png:
Dets: 67, GT: 61
Prec: 0.806, Rec: 0.885, F1: 0.844
TP: 54, FP: 13, FN: 7

13.png:
Dets: 215, GT: 220
Prec: 0.833, Rec: 0.814, F1: 0.823
TP: 179, FP: 36, FN: 41

14.png:
Dets: 123, GT: 117
Prec: 0.797, Rec: 0.838, F1: 0.817
TP: 98, FP: 25, FN: 19

15.png:
Dets: 47, GT: 50
Prec: 0.830, Rec: 0.780, F1: 0.804
TP: 39, FP: 8, FN: 11

16.png:
Dets: 38, GT: 46
Prec: 0.974, Rec: 0.804, F1: 0.881
TP: 37, FP: 1, FN: 9

17.png:
Dets: 18, GT: 21
Prec: 0.889, Rec: 0.762, F1: 0.821
TP: 16, FP: 2, FN: 5

18.png:
Dets: 40, GT: 38
Prec: 0.800, Rec: 0.842, F1: 0.821
TP: 32, FP: 8, FN: 6

19.png:
Dets: 109, GT: 119
Prec: 0.908, Rec: 0.832, F1: 0.868
TP: 99, FP: 10, FN: 20

2.png:
Dets: 60, GT: 60
Prec: 0.850, Rec: 0.850, F1: 0.850
TP: 51, FP: 9, FN: 